In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import squidpy as sq
import anndata as ad
import seaborn as sns
import networkx as nx
import pandas as pd
import json 
import sys
sys.path.append("../src/")
sys.path.append("/data/bionets/je30bery/truncated_graphs/src/")
from truncated_graphs import *

- create random spatial data
- connect nearest neighbors
- compute centrality scores
- for variable r, truncate graph by border r and compute centralities for everything inside, taking only nodes insight the new "window" into account
- calculate error of new and old centralities
- scatter

In [3]:
with open("../configs-ramses.json", "r") as f:
    configs = json.load(f)

In [7]:
borders = [1, 2, 3, 4, 5, 6, 7, 8]

type = "mibitof"
real_data, bounds = get_mibitof(configs['real_data'])

"""
n = 20**2
bounds = (0, 20)

type = "regular"
coordinates = generate_coordinates(n=n, bounds=bounds, type=type)
reg_original = create_anndata(coordinates, n_neighs=4, bounds=bounds)
reg_original, reg_truncated = fully_process(adata_original=reg_original, bounds=bounds, borders=borders)

type = "hexagonal"
coordinates = generate_coordinates(n=n, bounds=bounds, type=type)
hex_original = create_anndata(coordinates, n_neighs=6, bounds=bounds)
hex_original, hex_truncated = fully_process(adata_original=hex_original, bounds=bounds, borders=borders)

type = "normal"
coordinates = generate_coordinates(n=n, bounds=bounds, type=type)
normal_original = create_anndata(coordinates, n_neighs=6, bounds=bounds)
normal_original, normal_truncated = fully_process(adata_original=normal_original, bounds=bounds, borders=borders)
"""

'\nn = 20**2\nbounds = (0, 20)\n\ntype = "regular"\ncoordinates = generate_coordinates(n=n, bounds=bounds, type=type)\nreg_original = create_anndata(coordinates, n_neighs=4, bounds=bounds)\nreg_original, reg_truncated = fully_process(adata_original=reg_original, bounds=bounds, borders=borders)\n\ntype = "hexagonal"\ncoordinates = generate_coordinates(n=n, bounds=bounds, type=type)\nhex_original = create_anndata(coordinates, n_neighs=6, bounds=bounds)\nhex_original, hex_truncated = fully_process(adata_original=hex_original, bounds=bounds, borders=borders)\n\ntype = "normal"\ncoordinates = generate_coordinates(n=n, bounds=bounds, type=type)\nnormal_original = create_anndata(coordinates, n_neighs=6, bounds=bounds)\nnormal_original, normal_truncated = fully_process(adata_original=normal_original, bounds=bounds, borders=borders)\n'

In [9]:
mibitof_original, mibitof_truncated = fully_process(adata_original=real_data, bounds=bounds, borders=[1])

In [12]:
pearson = list()
kendall = list()
cosine = list()
measures = ["degree", "closeness", "betweenness", "harmonic"]
statistics = ["pearson correlation", "kendall's tau", "cosine distance"]
results = dict()

for i, adatas_truncated in enumerate([mibitof_truncated]): # reg_truncated, hex_truncated, normal_truncated
    results[i] = dict()
    for measure in measures:
        results[i][measure] = pd.DataFrame(index=borders)
        for statistic in statistics:
            vals = list()
            for border in borders:
                vals.append(adatas_truncated[border].uns[f"{measure} {statistic}"])
            results[i][measure][statistic.capitalize()] = vals    

ValueError: Length of values (1) does not match length of index (8)

In [ ]:
for i in results.keys():
    conc = pd.concat(results[i])
    conc.reset_index(inplace=True)
    conc.rename({"level_0": "Centrality measure", "level_1": "Border width"}, inplace=True, axis=1)
    results[i] = conc
conc = pd.concat(results)
conc.reset_index(inplace=True)

In [ ]:
conc.rename({"level_0": "Type"}, inplace=True, axis=1)
conc["Type"].replace({0: "Regular", 1: "Hexagonal", 2: "Normal"}, inplace=True)
conc.drop(["level_1"], axis=1)

In [ ]:
melted = pd.melt(conc, id_vars=["Type", "Centrality measure", "Border width", "level_1"], value_name="Value", var_name="Statistic")

In [ ]:
sns.relplot(melted, kind="line", y="Value", x="Border width", hue="Centrality measure", col="Statistic", row="Type")
plt.savefig("../result_plots/correlation_coefficients.png")

In [ ]:
border = 1
type = "normal"
measure = "harmonic"
plot_graphs_with_errors(normal_original, normal_truncated[border], border=border, measure=measure, bounds=bounds)
plt.savefig(f"../result_plots/{type}/{type}_{measure}_border={border}_n={n}.png")

In [ ]:
measure = "closeness"
truncated = normal_truncated
original = normal_original

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
pal = sns.color_palette("plasma", len(original))

for i, border in enumerate(truncated.keys()):
    ax.scatter(truncated[border].obs["distance_to_border"], [border] * len(truncated[border]), truncated[border].obs[f"{measure} error"], 
           color=np.array(pal)[truncated[border].obs_names.values.astype(int)], marker='o', alpha=0.5)

ax.set_xlabel('Distance to Closest Border')
ax.set_ylabel('Border Width')
ax.set_zlabel('Error')
ax.set_title(f'{measure.capitalize()} Measure Errors vs Distance to Border and Border Width')
ax.set_box_aspect(None, zoom=0.85)
plt.tight_layout()
plt.savefig(f"../result_plots/{type}/{measure}_by_border_on_{type}_grid_n={n}.png", bbox_inches="tight")